# Fooooooooooooooocus
#### Colab NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Colab](https://colab.research.google.com) 部署 Fooocus 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式。
2. 安装 Fooocus：根据环境配置安装 Fooocus 和运行环境。
3. 下载模型：下载可选列表中的模型。
4. 启动 Fooocus：启动 Fooocus，并显示访问地址。

## 提示
1. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
2. 其他功能有自定义模型下载等功能，根据自己的需求进行使用。

In [ ]:
#@title 👇 环境配置
INIT_CONFIG = 1
# 消息格式输出
def echo(msg):
    print(f":: {msg}")



# ARIA2
class ARIA2:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 下载器
    def aria2(self, url, path, filename):
        import os
        if not os.path.exists(path + "/" + filename):
            echo(f"开始下载 {filename} ，路径: {path}/{filename}")
            !aria2c --console-log-level=error -c -x 16 -s 16 "{url}" -d "{path}" -o "{filename}"
            if os.path.exists(path + "/" + filename) and not os.path.exists(path + "/" + filename + ".aria2"):
                echo(f"{filename} 下载完成")
            else:
                echo(f"{filename} 下载中断")
        else:
            if os.path.exists(path + "/" + filename + ".aria2"):
                echo(f"开始下载 {filename} ，路径: {path}/{filename}")
                !aria2c --console-log-level=error -c -x 16 -s 16 "{url}" -d "{path}" -o "{filename}"
                if os.path.exists(path + "/" + filename) and not os.path.exists(path + "/" + filename + ".aria2"):
                    echo(f"{filename} 下载完成")
                else:
                    echo(f"{filename} 下载中断")
            else:
                echo(f"{filename} 文件已存在，路径: {path}/{filename}")


    # 大模型下载
    def get_sd_model(self, url, filename):
        pass


    # lora下载
    def get_lora_model(self, url, filename):
        pass



# GIT
class GIT:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 检测要克隆的项目是否存在于指定路径
    def exists(self, addr=None, path=None, name=None):
        import os
        if addr is not None:
            if path is None and name is None:
                path = os.getcwd() + "/" + addr.split("/").pop().split(".git", 1)[0]
            elif path is None and name is not None:
                path = os.getcwd() + "/" + name
            elif path is not None and name is None:
                path = os.path.normpath(path) + "/" + addr.split("/").pop().split(".git", 1)[0]

        if os.path.exists(path):
            return True
        else:
            return False


    # 克隆项目
    def clone(self, addr, path=None, name=None):
        import os
        repo = addr.split("/").pop().split(".git", 1)[0]
        if not self.exists(addr, path, name):
            echo(f"开始下载 {repo}")
            if path is None and name is None:
                path = os.getcwd()
                name = repo
            elif path is not None and name is None:
                name = repo
            elif path is None and name is not None:
                path = os.getcwd()
            !git clone {addr} "{path}/{name}" --recurse-submodules
        else:
            echo(f"{repo} 已存在")



# TUNNEL
class TUNNEL:
    LOCALHOST_RUN = "localhost.run"
    REMOTE_MOE = "remote.moe"
    WORKSPACE = ""
    WORKFOLDER = ""
    PORT = ""


    def __init__(self, workspace, workfolder, port) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder
        self.PORT = port


    # ngrok内网穿透
    def ngrok(self, ngrok_token: str):
        from pyngrok import conf, ngrok
        conf.get_default().auth_token = ngrok_token
        conf.get_default().monitor_thread = False
        port = self.PORT
        ssh_tunnels = ngrok.get_tunnels(conf.get_default())
        if len(ssh_tunnels) == 0:
            ssh_tunnel = ngrok.connect(port, bind_tls=True)
            return ssh_tunnel.public_url
        else:
            return ssh_tunnels[0].public_url


    # cloudflare内网穿透
    def cloudflare(self):
        from pycloudflared import try_cloudflare
        port = self.PORT
        urls = try_cloudflare(port).tunnel
        return urls


    from typing import Union
    from pathlib import Path

    # 生成ssh密钥
    def gen_key(self, path: Union[str, Path]) -> None:
        import subprocess
        import shlex
        from pathlib import Path
        path = Path(path)
        arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
        args = shlex.split(arg_string)
        subprocess.run(args, check=True)
        path.chmod(0o600)


    # ssh内网穿透
    def ssh_tunnel(self, host: str) -> None:
        import subprocess
        import atexit
        import shlex
        import re
        import os
        from pathlib import Path
        from tempfile import TemporaryDirectory

        ssh_name = "id_rsa"
        ssh_path = Path(self.WORKSPACE) / ssh_name
        port = self.PORT

        tmp = None
        if not ssh_path.exists():
            try:
                self.gen_key(ssh_path)
            # write permission error or etc
            except subprocess.CalledProcessError:
                tmp = TemporaryDirectory()
                ssh_path = Path(tmp.name) / ssh_name
                self.gen_key(ssh_path)

        arg_string = f"ssh -R 80:127.0.0.1:{port} -o StrictHostKeyChecking=no -i {ssh_path.as_posix()} {host}"
        args = shlex.split(arg_string)

        tunnel = subprocess.Popen(
            args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)
        if tmp is not None:
            atexit.register(tmp.cleanup)

        tunnel_url = ""
        LOCALHOST_RUN = self.LOCALHOST_RUN
        lines = 27 if host == LOCALHOST_RUN else 5
        localhostrun_pattern = re.compile(r"(?P<url>https?://\S+\.lhr\.life)")
        remotemoe_pattern = re.compile(r"(?P<url>https?://\S+\.remote\.moe)")
        pattern = localhostrun_pattern if host == LOCALHOST_RUN else remotemoe_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")

            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                if lines == 27:
                    os.environ['LOCALHOST_RUN'] = tunnel_url
                    return tunnel_url
                else:
                    os.environ['REMOTE_MOE'] = tunnel_url
                    return tunnel_url
                # break
        else:
            echo(f"启动 {host} 内网穿透失败")


    # localhost.run穿透
    def localhost_run(self):
        urls = self.ssh_tunnel(self.LOCALHOST_RUN)
        return urls


    # remote.moe内网穿透
    def remote_moe(self):
        urls = self.ssh_tunnel(self.REMOTE_MOE)
        return urls


    # gradio内网穿透
    def gradio(self):
        import subprocess
        import shlex
        import atexit
        import re
        port = self.PORT
        cmd = f"gradio-tunneling --port {port}"
        cmd = shlex.split(cmd)
        tunnel = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)

        tunnel_url = ""
        lines = 5
        gradio_pattern = re.compile(r"(?P<url>https?://\S+\.gradio\.live)")
        pattern = gradio_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")
            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                return tunnel_url
        else:
            echo(f"启动 Gradio 内网穿透失败")


    # 启动内网穿透
    def start(self, ngrok=False, ngrok_token=None, cloudflare=False, remote_moe=False, localhost_run=False, gradio=False):
        if cloudflare is True or ngrok is True or ngrok_token is not None or remote_moe is True or localhost_run is True or gradio is True:
            echo("启动内网穿透")

        if cloudflare is True:
            cloudflare_url = self.cloudflare()
        else:
            cloudflare_url = None

        if ngrok is True and ngrok_token is not None:
            ngrok_url = self.ngrok(ngrok_token)
        else:
            ngrok_url = None

        if remote_moe is True:
            remote_moe_url = self.remote_moe()
        else:
            remote_moe_url = None

        if localhost_run is True:
            localhost_run_url = self.localhost_run()
        else:
            localhost_run_url = None

        if gradio is True:
            gradio_url = self.gradio()
        else:
            gradio_url = None

        echo("下方为访问地址")
        print("==================================================================================")
        echo(f"CloudFlare: {cloudflare_url}")
        echo(f"Ngrok: {ngrok_url}")
        echo(f"remote.moe: {remote_moe_url}")
        echo(f"localhost_run: {localhost_run_url}")
        echo(f"Gradio: {gradio_url}")
        print("==================================================================================")




# ENV
class ENV:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 准备ipynb笔记自身功能的依赖
    def prepare_env_depend(self, use_uv=False):
        pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"
        echo("安装自身组件依赖")
        if use_uv:
            !pip install uv {pip_mirror}
            !uv pip install pyngrok pycloudflared gradio-tunneling {pip_mirror} --system --quiet
        else:
            !pip install pyngrok pycloudflared gradio-tunneling {pip_mirror}

        !apt update
        !apt install aria2 ssh google-perftools -y


    # 安装pytorch和xformers
    def prepare_torch(self, torch_ver, xformers_ver, use_uv=False):
        pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"
        if use_uv:
            if torch_ver != "":
                echo("安装 PyTorch")
                !uv pip install {torch_ver} {pip_mirror} --system --quiet
            if xformers_ver != "":
                echo("安装 xFormers")
                !uv pip install {xformers_ver} {pip_mirror} --no-deps --system --quiet
        else:
            if torch_ver != "":
                echo("安装 PyTorch")
                !pip install {torch_ver} {pip_mirror}
            if xformers_ver != "":
                echo("安装 xFormers")
                !pip install {xformers_ver} {pip_mirror} --no-deps



    # 安装requirements.txt依赖
    def install_requirements(self, path, use_uv=False):
        import os
        pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"
        if os.path.exists(path):
            echo("安装依赖")
            if use_uv:
                !uv pip install -r "{path}" {pip_mirror} --system --quiet
            else:
                !pip install -r "{path}" {pip_mirror}
        else:
            echo("依赖文件路径为空")



    # 适用于colab的内存优化
    def tcmalloc_colab(self):
        echo("配置内存优化")
        import os
        aria2 = ARIA2(self.WORKSPACE, self.WORKFOLDER)
        path = self.WORKSPACE
        libtcmalloc_path = self.WORKSPACE + "/libtcmalloc_minimal.so.4"
        aria2.aria2("https://github.com/licyk/term-sd/releases/download/archive/libtcmalloc_minimal.so.4", path, "libtcmalloc_minimal.so.4")
        os.environ["LD_PRELOAD"] = libtcmalloc_path



# MANAGER
class MANAGER:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 清理ipynb笔记的输出
    def clear_up(self, opt):
        from IPython.display import clear_output
        clear_output(wait=opt)


    # 检查gpu是否可用
    def check_gpu(self):
        echo("检测 GPU 是否可用")
        import tensorflow as tf
        echo(f"TensorFlow 版本: {tf.__version__}")
        if tf.test.gpu_device_name():
            echo("GPU 可用")
        else:
            echo("GPU 不可用")
            raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")


    # 配置google drive
    def config_google_drive(self):
        echo("挂载 Google Drive")
        import os
        if not os.path.exists('/content/drive/MyDrive'):
            from google.colab import drive
            drive.mount('/content/drive')
            echo("Google Dirve 挂载完成")
        else:
            echo("Google Drive 已挂载")

        # 检测并创建输出文件夹
        if os.path.exists('/content/drive/MyDrive'):
            if not os.path.exists('/content/drive/MyDrive/fooocus_output'):
                echo("在 Google Drive 创建 fooocus_ouput 文件夹")
                !mkdir -p /content/drive/MyDrive/fooocus_output
        else:
            raise Exception("未挂载 Google Drive，请重新挂载后重试！")



# FOOOCUS
class FOOOCUS(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 7865)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 下载大模型
    def get_sd_model(self, url, filename = None):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/checkpoints"
        filename = url.split("/").pop() if filename is None else filename
        super().aria2(url, path, filename)


    # 下载lora模型
    def get_lora_model(self, url, filename = None):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/loras"
        filename = url.split("/").pop() if filename is None else filename
        super().aria2(url, path, filename)


    # 下载配置文件
    def install_config(self):
        path = self.WORKSPACE + "/" + self.WORKFOLDER
        echo("下载配置文件")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json", path + "/presets", "custom.json")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json", path, "config.txt")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json", path + "/language", "zh.json")


    # 预下载模型
    def pre_download_model(self, path):
        import os
        import json

        if os.path.exists(path):
            try:
                with open(path, "r", encoding="utf8") as file:
                    data = json.load(file)
                    echo("预下载 Fooocus 模型中")
            except Exception:
                # json 文件格式出现问题
                data = {}
        else:
            data = {}

        sd_model_list = data.get("checkpoint_downloads") if not isinstance(data.get("checkpoint_downloads"), type(None)) else {}
        lora_list = data.get("lora_downloads") if not isinstance(data.get("lora_downloads"), type(None)) else {}
        vae_list = data.get("vae_downloads") if not isinstance(data.get("vae_downloads"), type(None)) else {}
        sd_model_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "checkpoints")
        lora_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "loras")
        vae_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "vae")

        for i in sd_model_list:
            super().aria2(sd_model_list.get(i), sd_model_path, i)

        for i in lora_list:
            super().aria2(lora_list.get(i), lora_path, i)

        for i in vae_list:
            super().aria2(vae_list.get(i), vae_path, i)

        echo("预下载 Fooocus 模型完成")


    # 安装fooocus
    # 已知使用 uv 包管理器的缺陷: https://github.com/astral-sh/uv/blob/main/PIP_COMPATIBILITY.md#local-version-identifiers
    def install(self, torch_ver, xformers_ver, use_uv):
        import os
        req_file = os.path.join(self.WORKSPACE, self.WORKFOLDER, "requirements_versions.txt")
        config_file = os.path.join(self.WORKSPACE, self.WORKFOLDER, "presets", "custom.json")
        self.check_gpu()
        self.prepare_env_depend(use_uv)
        self.clone("https://github.com/lllyasviel/Fooocus", self.WORKSPACE)
        os.chdir(os.path.join(self.WORKSPACE, self.WORKFOLDER))
        self.prepare_torch(torch_ver, xformers_ver, use_uv)
        self.install_requirements(req_file, use_uv)
        if use_uv: # 纠正pytorch版本
            self.prepare_torch(torch_ver, xformers_ver, use_uv)

        self.install_config()
        self.tcmalloc_colab()
        self.pre_download_model(config_file)


#######################################################


#@markdown ### 核心组件版本：
#@markdown #### PyTorch：
pytorch_ver = "torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121"  #@param {type:"string"}
#@markdown #### xFormers：
xformers_ver = "xformers==0.0.27"  #@param {type:"string"}
#@markdown ---
#@markdown - 使用 uv 作为 Python 包管理器
use_uv = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### 选择内网穿透的方式（对应下方的勾选框）：
#@markdown - 使用 remote.moe 内网穿透
#@markdown - 使用 localhost.run 内网穿透
#@markdown - 使用 CloudFlare 内网穿透
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
#@markdown - 使用 Gradio 内网穿透
use_remote_moe = True #@param {type:"boolean"}
use_localhost_run = True #@param {type:"boolean"}
use_cloudflare = True #@param {type:"boolean"}
use_ngrok = False #@param {type:"boolean"}
use_gradio = True #@param {type:"boolean"}
#@markdown ### Ngrok Token：
ngrok_token = ""  #@param {type:"string"}


#######################################################


fooocus = FOOOCUS("/content","Fooocus")
fooocus.install(pytorch_ver, xformers_ver, use_uv)
fooocus.clear_up(False)
echo("Fooocus 运行环境配置完成")

In [ ]:
#@title 👇 下载模型（可选）

try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

# 模型下载
echo("下载模型中")
#@markdown 选择下载的模型：
##############################


sd_xl_base = False  #@param {type:"boolean"}
if sd_xl_base:
    fooocus.get_sd_model("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors")

anima_pencil_xl = False  #@param {type:"boolean"}
if anima_pencil_xl:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323674", "anima_pencil_xl.safetensors")

bluePencilXL  = False  #@param {type:"boolean"}
if bluePencilXL:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323375", "bluePencilXL_v050.safetensors")

AnythingXL_xl = False  #@param {type:"boolean"}
if AnythingXL_xl:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/AnythingXL_xl.safetensors")

abyssorangeXLElse_v10 = False  #@param {type:"boolean"}
if abyssorangeXLElse_v10:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/abyssorangeXLElse_v10.safetensors")

CounterfeitXL  = False  #@param {type:"boolean"}
if CounterfeitXL:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors")

animeIllustDiffusion  = False  #@param {type:"boolean"}
if animeIllustDiffusion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors")

nekorayxl = False  #@param {type:"boolean"}
if nekorayxl:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors")

animagine_XL_3 = False  #@param {type:"boolean"}
if animagine_XL_3:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors")

animagine_XL_3_1 = True  #@param {type:"boolean"}
if animagine_XL_3_1:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors")

heartOfAppleXL_v20 = False  #@param {type:"boolean"}
if heartOfAppleXL_v20:
    fooocus.get_sd_model("https://civitai.com/api/download/models/337306", "heartOfAppleXL_v30.safetensors")

heartOfAppleXL_v30 = False  #@param {type:"boolean"}
if heartOfAppleXL_v30:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors")

holodayo_xl_21 = False  #@param {type:"boolean"}
if holodayo_xl_21:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors")

kivotos_xl_20 = False  #@param {type:"boolean"}
if kivotos_xl_20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors")

sd_xl_anime_V52 = False  #@param {type:"boolean"}
if sd_xl_anime_V52:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_anime_V52.safetensors")

kohakuXLGamma_rev1  = False  #@param {type:"boolean"}
if kohakuXLGamma_rev1:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-gamma/resolve/main/kohaku-xl-gamma-rev1.safetensors")

kohakuXLBeta_beta7  = False  #@param {type:"boolean"}
if kohakuXLBeta_beta7:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLBeta_beta7.safetensors")

Kohaku_XL_Delta = False  #@param {type:"boolean"}
if Kohaku_XL_Delta:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Delta/resolve/main/kohaku-xl-delta-rev1.safetensors")

kohakuXL_Epsilon = False  #@param {type:"boolean"}
if kohakuXL_Epsilon:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon/resolve/main/kohaku-xl-epsilon-rev1.safetensors")

kohakuXL_Epsilon_rev2 = False  #@param {type:"boolean"}
if kohakuXL_Epsilon_rev2:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev2/resolve/main/kohaku-xl-epsilon-rev2.safetensors")

kohakuXL_Epsilon_rev3 = False  #@param {type:"boolean"}
if kohakuXL_Epsilon_rev3:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev3/resolve/main/kohaku-xl-epsilon-rev3.safetensors")

kohakuXL_Zeta = True  #@param {type:"boolean"}
if kohakuXL_Zeta:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Zeta/resolve/main/kohaku-xl-zeta.safetensors")

starryXLV52 = False  #@param {type:"boolean"}
if starryXLV52:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors")

BArtstyleDB = False  #@param {type:"boolean"}
if BArtstyleDB:
    fooocus.get_sd_model("https://civitai.com/api/download/models/299095", "BArtstyleDB_XL.safetensors")

BArtstyleDB_3 = False  #@param {type:"boolean"}
if BArtstyleDB_3:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBlueArchiveFlatCelluloidStyle_xlv3.safetensors")

pony_v6 = False  #@param {type:"boolean"}
if pony_v6:
    fooocus.get_sd_model("https://civitai.com/api/download/models/290640", "ponyDiffusionV6XL_v6StartWithThisOne.safetensors")

pdForAnime_v20 = False  #@param {type:"boolean"}
if pdForAnime_v20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors")

tPonynai3_v51 = False  #@param {type:"boolean"}
if tPonynai3_v51:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tPonynai3_v51WeightOptimized.safetensors")

omegaPonyXLAnime_v20 = False  #@param {type:"boolean"}
if omegaPonyXLAnime_v20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors")


##############################
fooocus.clear_up(False)
echo("模型下载完成")

In [ ]:
#@title 👇 启动 Fooocus（图片保存在 Google Drive）
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

echo("启动 Fooocus")
import os
os.chdir("/content/Fooocus")
fooocus.config_google_drive()
fooocus.tun.start(ngrok=use_ngrok, ngrok_token=ngrok_token, cloudflare=use_cloudflare, remote_moe=use_remote_moe, localhost_run=use_localhost_run, gradio=False)
if use_gradio:
    !python /content/Fooocus/launch.py --preset custom --language zh --disable-offload-from-vram --async-cuda-allocation --share --disable-analytics
else:
    !python /content/Fooocus/launch.py --preset custom --language zh --disable-offload-from-vram --async-cuda-allocation --disable-analytics
fooocus.clear_up(False)
echo("已关闭 Fooocus")

### ✨ 其他功能

In [ ]:
#@title 👇 自定义模型下载
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

#@markdown ### 选择模型种类：
#@markdown - Stable Diffusion（大模型）模型
#@markdown - LoRA模型
model_type = "LoRA" # @param ["Stable Diffusion", "LoRA"]
#@markdown ###填写模型的下载链接：
model_url = "https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/CoolFlatColor.safetensors"  #@param {type:"string"}
#@markdown ###填写模型的名称（包括后缀名）：
model_name = "CoolFlatColor.safetensors"  #@param {type:"string"}

if model_type == "Stable Diffusion":
    fooocus.get_sd_model(model_url, model_name)
elif model_type == "LoRA":
    fooocus.get_lora_model(model_url, model_name)

In [ ]:
#@title 👇 启动Fooocus（图片保存在 Colab）
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")
echo("启动 Fooocus")
import os
os.chdir("/content/Fooocus")
fooocus.tun.start(ngrok=use_ngrok, ngrok_token=ngrok_token, cloudflare=use_cloudflare, remote_moe=use_remote_moe, localhost_run=use_localhost_run, gradio=False)
if use_gradio:
    !python /content/Fooocus/launch.py --preset custom --language zh --disable-offload-from-vram --async-cuda-allocation --share
else:
    !python /content/Fooocus/launch.py --preset custom --language zh --disable-offload-from-vram --async-cuda-allocation
fooocus.clear_up(False)
echo("已关闭 Fooocus")

In [ ]:
#@title 👇挂载 Google Drive 并备份图片
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")
fooocus.config_google_drive()
echo("备份图片中")
!cp -r /content/drive/MyDrive/fooocus_output/* /content/drive_/MyDrive/fooocus_output
echo("备份完成")

In [ ]:
#@title 👇远程共享方式修改
#@markdown ### 选择内网穿透的方式（对应下方的勾选框）：
#@markdown - 使用 remote.moe 内网穿透
#@markdown - 使用 localhost.run 内网穿透
#@markdown - 使用 CloudFlare 内网穿透
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
#@markdown - 使用 Gradio 内网穿透
use_remote_moe = True #@param {type:"boolean"}
use_localhost_run = True #@param {type:"boolean"}
use_cloudflare = True #@param {type:"boolean"}
use_ngrok = False #@param {type:"boolean"}
use_gradio = True #@param {type:"boolean"}
#@markdown ### Ngrok Token：
ngrok_token = ""  #@param {type:"string"}

echo("设置共享方式完成")